In [1]:
import os
import numpy as np
import warnings

import matplotlib.pyplot as plt

from PIL import Image, ImageFile
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dropout

from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers.schedules import ExponentialDecay

##### Get the Data and Load them and labels

In [2]:
# allow partially corrupted images
ImageFile.LOAD_TRUNCATED_IMAGES = True
warnings.filterwarnings("ignore", category=UserWarning, module="PIL.TiffImagePlugin")

data_dir = '/Users/kaunli/Desktop/School/Machine Learning/Project/WasteClassData'
image_size = (224, 224)
categories = ['Hazardous', 'Non-Recyclable', 'Organic', 'Recyclable']
num_classes = len(categories)
allowed_extensions = ('.jpg', '.jpeg', '.png')

In [3]:
# load the images
images = []
labels = []

for idx, main_category in enumerate(categories):
    main_path = os.path.join(data_dir, main_category, main_category)
    if not os.path.exists(main_path):
        print(f"Path not found: {main_path}")
        continue
    for subfolder in os.listdir(main_path):
        subfolder_path = os.path.join(main_path, subfolder)
        if not os.path.isdir(subfolder_path):
            continue
        for img_file in os.listdir(subfolder_path):
            if img_file.startswith('.') or not img_file.lower().endswith(allowed_extensions):
                continue
            try:
                img_path = os.path.join(subfolder_path, img_file)
                img = Image.open(img_path).convert('RGBA')  # handle transparency
                img = img.convert('RGB')  # convert to RGB
                img = img.resize(image_size)
                img_array = np.array(img, dtype='float32')
                images.append(img_array)
                labels.append(idx)
            except Exception as e:
                print(f"Skipping corrupted or unsupported file: {img_path} ({e})")

images = np.array(images)
labels = np.array(labels)

print("Loaded images:", images.shape)
print("Loaded labels:", labels.shape)

Loaded images: (2884, 224, 224, 3)
Loaded labels: (2884,)


##### Split train, validation, and test sets with 80/10/10

In [4]:
X_train, X_temp, y_train, y_temp = train_test_split(
    images, labels, test_size=0.2, random_state=42, stratify=labels
)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp
)


print("Train:", X_train.shape, y_train.shape)
print("Validation:", X_val.shape, y_val.shape)
print("Test:", X_test.shape, y_test.shape)

Train: (2307, 224, 224, 3) (2307,)
Validation: (288, 224, 224, 3) (288,)
Test: (289, 224, 224, 3) (289,)


##### One Hot Encoding

In [5]:
y_train = to_categorical(y_train, num_classes)
y_val = to_categorical(y_val, num_classes)
y_test = to_categorical(y_test, num_classes)

##### Compute Class Weights

In [6]:
from sklearn.utils import class_weight
from sklearn.utils.class_weight import compute_class_weight

class_weights = class_weight.compute_class_weight(
    'balanced',
    classes=np.unique(np.argmax(y_train, axis=1)),
    y=np.argmax(y_train, axis=1)
)
class_weights_dict = dict(enumerate(class_weights))
print("Class weights:", class_weights_dict)

Class weights: {0: 0.7741610738255034, 1: 1.1220817120622568, 2: 1.0944022770398483, 3: 1.1070057581573896}


##### Data Augmentation

In [7]:
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    brightness_range=[0.8, 1.2],
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

train_generator = train_datagen.flow(X_train, y_train, batch_size=32)
val_generator = val_datagen.flow(X_val, y_val, batch_size=32)
test_generator = test_datagen.flow(X_test, y_test, batch_size=32, shuffle=False)

##### CNN Models

##### Baseline CNN (from scratch)

In [8]:
warnings.filterwarnings("ignore", category=UserWarning, module='keras')

baseline_model = Sequential([
    Input(shape=(224, 224, 3)),
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])

# compile
baseline_model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

2025-11-12 15:34:53.524121: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M3
2025-11-12 15:34:53.524409: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2025-11-12 15:34:53.524423: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.92 GB
2025-11-12 15:34:53.524716: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-11-12 15:34:53.524729: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [9]:
# train the model
history = baseline_model.fit(
    train_generator,
    steps_per_epoch=len(X_train) // 32,
    epochs=20,
    validation_data=val_generator,
    validation_steps=len(X_val) // 32
)
# evaluate the model
test_loss, test_accuracy = baseline_model.evaluate(test_generator, steps=len(X_test) // 32)
print(f'Test accuracy: {test_accuracy:.4f}')

Epoch 1/20


2025-11-12 15:34:54.265744: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


72/72 ━━━━━━━━━━━━━━━━━━━━ 12s 158ms/step - accuracy: 0.2554 - loss: 323.9544 - val_accuracy: 0.3125 - val_loss: 1.3861
Epoch 2/20
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4062 - loss: 1.4425 - val_accuracy: 0.3125 - val_loss: 1.3858
Epoch 3/20
72/72 ━━━━━━━━━━━━━━━━━━━━ 11s 150ms/step - accuracy: 0.3121 - loss: 1.3900 - val_accuracy: 0.3056 - val_loss: 1.3783
Epoch 4/20
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.1875 - loss: 1.4059 - val_accuracy: 0.3056 - val_loss: 1.3784
Epoch 5/20
72/72 ━━━━━━━━━━━━━━━━━━━━ 11s 147ms/step - accuracy: 0.3266 - loss: 1.3878 - val_accuracy: 0.3264 - val_loss: 1.4151
Epoch 6/20
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.2812 - loss: 1.4135 - val_accuracy: 0.3333 - val_loss: 1.4197
Epoch 7/20
72/72 ━━━━━━━━━━━━━━━━━━━━ 11s 145ms/step - accuracy: 0.3288 - loss: 1.3810 - val_accuracy: 0.3160 - val_loss: 1.3632
Epoch 8/20
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.1562 - loss: 1.4751 - val_accuracy: 0.3160 - val_lo

##### VGG16 Model

In [10]:
# define the VGG16 model
vgg_base = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
vgg_base.trainable = False  # freeze the base model

x = GlobalAveragePooling2D()(vgg_base.output)
x = BatchNormalization()(x)
x = Dense(128, activation='relu', kernel_regularizer='l2')(x)
x = Dropout(0.5)(x)
output = Dense(num_classes, activation='softmax')(x)

# create the model
model = Model(inputs=vgg_base.input, outputs=output)

In [11]:
# define callbacks and learning rate scheduler
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
checkpoint = ModelCheckpoint('best_vgg16_ecosort.keras', monitor='val_accuracy', save_best_only=True)

# learning rate schedule
initial_lr = 2e-5
lr_schedule = ExponentialDecay(
    initial_learning_rate=initial_lr,
    decay_steps=len(train_generator),
    decay_rate=0.9,
    staircase=True
)

In [ ]:
# compile the model
model.compile(
    optimizer=Adam(learning_rate=lr_schedule),
    loss=CategoricalCrossentropy(label_smoothing=0.1),
    metrics=['accuracy']
)
# train the model
history = model.fit(
    train_generator,
    epochs=20,
    validation_data=val_generator,
    class_weight=class_weights_dict,
    callbacks=[early_stop, checkpoint]
)

# evaluate the model
test_loss, test_accuracy = model.evaluate(test_generator, steps=len(X_test) // 32)
print(f'Test accuracy: {test_accuracy:.4f}')

Epoch 1/20
73/73 ━━━━━━━━━━━━━━━━━━━━ 26s 349ms/step - accuracy: 0.2436 - loss: 4.8097 - val_accuracy: 0.2743 - val_loss: 5.0203
Epoch 2/20
73/73 ━━━━━━━━━━━━━━━━━━━━ 27s 365ms/step - accuracy: 0.2865 - loss: 4.5961 - val_accuracy: 0.2882 - val_loss: 4.4901
Epoch 3/20
73/73 ━━━━━━━━━━━━━━━━━━━━ 28s 384ms/step - accuracy: 0.3082 - loss: 4.5246 - val_accuracy: 0.3090 - val_loss: 4.2637
Epoch 4/20
73/73 ━━━━━━━━━━━━━━━━━━━━ 28s 378ms/step - accuracy: 0.3195 - loss: 4.3985 - val_accuracy: 0.3472 - val_loss: 4.1060
Epoch 5/20
73/73 ━━━━━━━━━━━━━━━━━━━━ 28s 384ms/step - accuracy: 0.3242 - loss: 4.3447 - val_accuracy: 0.3681 - val_loss: 4.0210
Epoch 6/20
73/73 ━━━━━━━━━━━━━━━━━━━━ 28s 377ms/step - accuracy: 0.3338 - loss: 4.3214 - val_accuracy: 0.3889 - val_loss: 3.9347
Epoch 7/20
73/73 ━━━━━━━━━━━━━━━━━━━━ 30s 409ms/step - accuracy: 0.3446 - loss: 4.2230 - val_accuracy: 0.3958 - val_loss: 3.8786
Epoch 8/20
73/73 ━━━━━━━━━━━━━━━━━━━━ 35s 473ms/step - accuracy: 0.3533 - loss: 4.1951 - val_accu

In [ ]:
vgg_base.trainable = True
for layer in vgg_base.layers[:-15]:
    layer.trainable = False

# recompile with smaller LR
fine_tune_lr = ExponentialDecay(
    initial_learning_rate=1e-5,
    decay_steps=len(train_generator),
    decay_rate=0.9,
    staircase=True
)

In [ ]:
# compile the model
model.compile(
    optimizer=Adam(learning_rate=fine_tune_lr),
    loss=CategoricalCrossentropy(label_smoothing=0.1),
    metrics=['accuracy']
)

# train the model
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=val_generator,
    class_weight=class_weights_dict,
    callbacks=[early_stop, checkpoint]
)

# evaluate the model
test_loss, test_accuracy = model.evaluate(test_generator, steps=len(X_test) // 32)
print(f"Fine-tuned Test Accuracy: {test_accuracy:.4f}")

##### Plot Training vs Validation Accuracy/Loss Curve

In [ ]:
# Plot training & validation accuracy values
plt.figure(figsize=(10, 4))

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy Over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)

# Plot training & validation loss values
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss Over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()